In [30]:
import os
import pandas as pd
import numpy as np
import random
import json

In [31]:
data_set_list=('FrcSub','Math1', 'Math2')
data_set_name=data_set_list[2]

base_dir='../data/'
if data_set_name == 'FrcSub':
    read_dir=os.path.join(base_dir,'Math2015/FrcSub/')
    save_dir=os.path.join(base_dir,'frcSub/')
elif data_set_name == 'Math1':
    read_dir=os.path.join(base_dir,'Math2015/Math1/')
    save_dir=os.path.join(base_dir,'math1/')
elif data_set_name == 'Math2':
    read_dir=os.path.join(base_dir,'Math2015/Math2/')
    save_dir=os.path.join(base_dir,'math2/')
    
X = np.loadtxt(os.path.join(read_dir, 'data.txt'))
Q = np.loadtxt(os.path.join(read_dir, 'q.txt'))


In [32]:
def split_train_test(X,test_ratio=0.2):
    stu_num=X.shape[0]
    prob_num=X.shape[1]
    train_data=[[],[],[]]
    test_data=[[],[],[]]
    stu_problem=np.array(range(1,prob_num+1))
    for stu in range(1,stu_num+1):
        stu_correct=X[stu-1,:]    
        test_index=random.sample(list(range(prob_num)),int(np.ceil(prob_num*test_ratio)))
        train_index=list(set(range(prob_num))-set(test_index))

        train_data[0].extend([stu]*len(train_index))
        train_data[1].extend(stu_problem[train_index])
        train_data[2].extend(stu_correct[train_index])

        test_data[0].extend([stu]*len(test_index))
        test_data[1].extend(stu_problem[test_index])
        test_data[2].extend(stu_correct[test_index])
    
    train=pd.DataFrame({'user_id':np.array(train_data[0],dtype='int'),
                        'item_id':np.array(train_data[1],dtype='int'),
                        'score':train_data[2]})
    test=pd.DataFrame({'user_id':np.array(test_data[0],dtype='int'),
                       'item_id':np.array(test_data[1],dtype='int'),
                       'score':test_data[2]})
    return train,test

In [33]:
def format_Q_matrix(Q):
    problem_list=np.array(range(1,Q.shape[0]+1))
    item_data=pd.DataFrame(columns=['item_id','knowledge_code'])
    counter=0
    for problem in range(1,len(problem_list)+1):
        item_data.loc[counter,'item_id']=str(problem)
        item_data.loc[counter,'knowledge_code']=list(np.argwhere(Q[problem-1,:]==1).reshape(-1)+1)
        counter+=1
    return item_data

In [34]:
def read_probdesc(filepath):
    prob_type = np.loadtxt(filepath, dtype=int, delimiter='\t',
                           converters={0: lambda x: int(x) - 1, 1: lambda s: s == b'Obj'}, skiprows=1)
    obj_prob_index, sub_prob_index = prob_type[prob_type[:, 1] == 1][:, 0], prob_type[prob_type[:, 1] == 0][:, 0]
    return prob_type, obj_prob_index, sub_prob_index

In [35]:
# type of problems
problems, obj_prob_index, sub_prob_index = read_probdesc(read_dir+"problemdesc.txt")
np.savetxt(save_dir+"obj_prob_index.csv", obj_prob_index, delimiter=',', fmt='%d')
np.savetxt(save_dir+"sub_prob_index.csv", sub_prob_index, delimiter=',', fmt='%d')

In [36]:
test_ratio=0.2

train,test=split_train_test(X,test_ratio)

train.to_csv(save_dir+'train.csv',index=False)
test.to_csv(save_dir+'test.csv',index=False)

In [37]:
#将习题保存为 json 格式

train_dict=train.loc[:,['user_id','item_id']]-1
train_dict.loc[:,'score']=train.loc[:,'score']
train_dict=list(train_dict.T.to_dict('dict').values())
train_dict=[{'user_id':int(x['user_id']),'item_id':int(x['item_id']),'score':x['score']} for x in train_dict]

test_dict=test.loc[:,['user_id','item_id']]-1
test_dict.loc[:,'score']=test.loc[:,'score']
test_dict=list(test_dict.T.to_dict('dict').values())
test_dict=[{'user_id':int(x['user_id']),'item_id':int(x['item_id']),'score':x['score']} for x in test_dict]

with open(save_dir+'train.json', 'w', encoding='utf8') as file:
    json.dump(train_dict, file, indent=4, ensure_ascii=False)
with open(save_dir+'test.json', 'w', encoding='utf8') as file:
    json.dump(test_dict, file, indent=4, ensure_ascii=False)

In [38]:
item=format_Q_matrix(Q)

item.to_csv(save_dir+'item.csv',index=False)

In [39]:
np.savetxt(save_dir+'q_m.csv', Q,  delimiter=',', fmt='%d')